In [1]:
from __future__ import print_function, absolute_import
import argparse
import os.path as osp
import os
import random
import numpy as np
import sys


from sklearn.cluster import DBSCAN
# from sklearn.preprocessing import normalize

import torch
from torch import nn
from torch.backends import cudnn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import Parameter
# from torch.nn import init
#
# from UDAsbs.utils.rerank import compute_jaccard_dist

from UDAsbs import datasets, sinkhornknopp as sk
from UDAsbs import models
from UDAsbs.trainers import DbscanBaseTrainer
from UDAsbs.evaluators import Evaluator, extract_features
from UDAsbs.utils.data import IterLoader
from UDAsbs.utils.data import transforms as T
from UDAsbs.utils.data.sampler import RandomMultipleGallerySampler
from UDAsbs.utils.data.preprocessor import Preprocessor
from UDAsbs.utils.logging import Logger
from UDAsbs.utils.serialization import load_checkpoint, save_checkpoint#, copy_state_dict
from UDAsbs.models.dsbn import convert_dsbn
from UDAsbs.memorybank.NCEAverage import onlinememory
from UDAsbs.utils.faiss_rerank import compute_jaccard_distance
# import ipdb

parser = argparse.ArgumentParser(description="MMT Training")
# data
parser.add_argument('-st', '--dataset-source', type=str, default='market1501',
                    choices=datasets.names())
parser.add_argument('-tt', '--dataset-target', type=str, default='dukemtmc',
                    choices=datasets.names())
parser.add_argument('-b', '--batch-size', type=int, default=64)
parser.add_argument('-j', '--workers', type=int, default=16)
parser.add_argument('--choice_c', type=int, default=0)
parser.add_argument('--num-clusters', type=int, default=700)
parser.add_argument('--ncs', type=str, default='60')
parser.add_argument('--k1', type=int, default=30,
                    help="hyperparameter for jaccard distance")
parser.add_argument('--k2', type=int, default=6,
                    help="hyperparameter for jaccard distance")
parser.add_argument('--height', type=int, default=256,
                    help="input height")
parser.add_argument('--width', type=int, default=128,
                    help="input width")
parser.add_argument('--num-instances', type=int, default=4,
                    help="each minibatch consist of "
                         "(batch_size // num_instances) identities, and "
                         "each identity has num_instances instances, "
                         "default: 0 (NOT USE)")
# model
parser.add_argument('-a', '--arch', type=str, default='ResNet50',
                    choices=models.names())
parser.add_argument('--features', type=int, default=0)
parser.add_argument('--dropout', type=float, default=0)
# optimizer
parser.add_argument('--lr', type=float, default=0.00035,
                    help="learning rate of new parameters, for pretrained "
                         "parameters it is 10 times smaller than this")
parser.add_argument('--momentum', type=float, default=0.9)
parser.add_argument('--alpha', type=float, default=0.999)
parser.add_argument('--moving-avg-momentum', type=float, default=0)
parser.add_argument('--weight-decay', type=float, default=5e-4)
parser.add_argument('--soft-ce-weight', type=float, default=0.5)
parser.add_argument('--soft-tri-weight', type=float, default=0.8)
parser.add_argument('--epochs', type=int, default=200)
parser.add_argument('--iters', type=int, default=200)
parser.add_argument('--lambda-value', type=float, default=0)
# training configs
parser.add_argument('--rr-gpu', action='store_true',
                    help="use GPU for accelerating clustering")
# parser.add_argument('--init-1', type=str, default='logs/personxTOpersonxval/resnet_ibn50a-pretrain-1_gem_RA//model_best.pth.tar', metavar='PATH')
parser.add_argument('--init-1', type=str,
                    default='logs/market1501TOdukemtmc/market_pretrain_resnet/model_best.pth.tar',
                    metavar='PATH')
parser.add_argument('--seed', type=int, default=1)
parser.add_argument('--print-freq', type=int, default=50)
parser.add_argument('--eval-step', type=int, default=5)
parser.add_argument('--n-jobs', type=int, default=16)
# path
working_dir = osp.dirname(osp.abspath(''))
parser.add_argument('--data-dir', type=str, metavar='PATH',
                    default=osp.join(working_dir, '/home/jun/ReID_Dataset'))
parser.add_argument('--logs-dir', type=str, metavar='PATH',
                    default=osp.join(working_dir, 'logs/dbscan-market1501TOdukemtmc/hrnet_sbs_mb'))
parser.add_argument('--lambda-tri', type=float, default=1.0)
parser.add_argument('--lambda-reg', type=float, default=1.0)
parser.add_argument('--lambda-ct', type=float, default=0.05)
parser.add_argument('--uncer-mode', type=float, default=0)#0 mean 1 max 2 min
print("======mmt_train_dbscan_self-labeling=======")

2.3.0
======mmt_train_dbscan_self-labeling=======


In [2]:
start_epoch = best_mAP = 0

def get_data(name, data_dir, l=1):
    root = osp.join(data_dir)

    dataset = datasets.create(name, root, l)

    label_dict = {}
    for i, item_l in enumerate(dataset.train):
        # dataset.train[i]=(item_l[0],0,item_l[2])
        if item_l[1] in label_dict:
            label_dict[item_l[1]].append(i)
        else:
            label_dict[item_l[1]] = [i]

    return dataset, label_dict


def get_train_loader(dataset, height, width, choice_c, batch_size, workers,
                     num_instances, iters, trainset=None):
    normalizer = T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    train_transformer = T.Compose([
        T.Resize((height, width), interpolation=3),
        T.RandomHorizontalFlip(),
        T.Pad(10),
        T.RandomCrop((height, width)),
        T.ToTensor(),
        normalizer,
        T.RandomErasing(probability=0.5, mean=[0.596, 0.558, 0.497])
    ])

    train_set = trainset #dataset.train if trainset is None else trainset
    rmgs_flag = num_instances > 0
    if rmgs_flag:
        sampler = RandomMultipleGallerySampler(train_set, num_instances, choice_c)
    else:
        sampler = None

    train_loader = IterLoader(
        DataLoader(Preprocessor(train_set, root=dataset.images_dir,
                                transform=train_transformer, mutual=True),
                   batch_size=batch_size, num_workers=workers, sampler=sampler,
                   shuffle=not rmgs_flag, pin_memory=True, drop_last=True), length=iters)

    return train_loader


def get_test_loader(dataset, height, width, batch_size, workers, testset=None):
    normalizer = T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

    test_transformer = T.Compose([
        T.Resize((height, width), interpolation=3),
        T.ToTensor(),
        normalizer
    ])

    if (testset is None):
        testset = list(set(dataset.query) | set(dataset.gallery))

    test_loader = DataLoader(
        Preprocessor(testset, root=dataset.images_dir, transform=test_transformer),
        batch_size=batch_size, num_workers=workers,
        shuffle=False, pin_memory=True)

    return test_loader




def copy_state_dict(state_dict, model, strip=None):
    tgt_state = model.state_dict()
    copied_names = set()
    for name, param in state_dict.items():
        name = name.replace('module.', '')
        if strip is not None and name.startswith(strip):
            name = name[len(strip):]
        if name not in tgt_state:
            continue
        if isinstance(param, Parameter):
            param = param.data
        if param.size() != tgt_state[name].size():
            print('mismatch:', name, param.size(), tgt_state[name].size())
            continue
        tgt_state[name].copy_(param)
        copied_names.add(name)

    missing = set(tgt_state.keys()) - copied_names
    if len(missing) > 0:
        print("missing keys in state_dict:", missing)

    return model

def create_model(args, ncs, wopre=False):
    model_1 = models.create(args.arch, num_features=args.features, dropout=args.dropout,
                            num_classes=ncs)

    model_1_ema = models.create(args.arch, num_features=args.features, dropout=args.dropout,
                                num_classes=ncs)
    if not wopre:

        initial_weights = load_checkpoint(args.init_1)
        copy_state_dict(initial_weights['state_dict'], model_1)
        copy_state_dict(initial_weights['state_dict'], model_1_ema)
        print('load pretrain model:{}'.format(args.init_1))

    # adopt domain-specific BN
    convert_dsbn(model_1)
    convert_dsbn(model_1_ema)
    model_1.cuda()
    model_1_ema.cuda()
    model_1 = nn.DataParallel(model_1)
    model_1_ema = nn.DataParallel(model_1_ema)

    for i, cl in enumerate(ncs):
        exec('model_1_ema.module.classifier{}_{}.weight.data.copy_(model_1.module.classifier{}_{}.weight.data)'.format(i,cl,i,cl))

    return model_1, None, model_1_ema, None


class Optimizer:
    def __init__(self, target_label, m, dis_gt, t_loader,N, hc=3, ncl=None,  n_epochs=200,
                 weight_decay=1e-5, ckpt_dir='/',fc_len=3500):
        self.num_epochs = n_epochs
        self.momentum = 0.9
        self.weight_decay = weight_decay
        self.checkpoint_dir = ckpt_dir
        self.N=N
        self.resume = True
        self.checkpoint_dir = None
        self.writer = None
        # model stuff
        self.hc = len(ncl)#10
        self.K = ncl#3000
        self.K_c =[fc_len for _ in range(len(ncl))]
        self.model = m
        self.dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.L = [torch.LongTensor(target_label[i]).to(self.dev) for i in range(len(self.K))]
        self.nmodel_gpus = 4#len()
        self.pseudo_loader = t_loader#torch.utils.data.DataLoader(t_loader,batch_size=256)
        # can also be DataLoader with less aug.
        self.train_loader = t_loader
        self.lamb = 25#args.lamb # the parameter lambda in the SK algorithm
        self.cpu=True
        self.dis_gt=dis_gt
        dtype_='f64'
        if dtype_ == 'f32':
            self.dtype = torch.float32 if not self.cpu else np.float32
        else:
            self.dtype = torch.float64 if not self.cpu else np.float64

        self.outs = self.K
        # activations of previous to last layer to be saved if using multiple heads.
        self.presize =  2048#4096 #

    def optimize_labels(self):
        if self.cpu:
            sk.cpu_sk(self)
        else:
            sk.gpu_sk(self)

        # save Label-assignments: optional
        # torch.save(self.L, os.path.join(self.checkpoint_dir, 'L', str(niter) + '_L.gz'))

        # free memory
        data = 0
        self.PS = 0

        return self.L

import collections


def func(x, a, b, c):
    return a * np.exp(-b * x) + c

def write_sta_im(train_loader):
    label2num=collections.defaultdict(int)
    save_label=[]
    for x in train_loader:
        label2num[x[1]]+=1
        save_label.append(x[1])
    labels=sorted(label2num.items(),key=lambda item:item[1])[::-1]
    num = [j for i, j in labels]
    distribution = np.array(num)/len(train_loader)

    return num,save_label
def print_cluster_acc(label_dict,target_label_tmp):
    num_correct = 0
    for pid in label_dict:
        pid_index = np.asarray(label_dict[pid])
        pred_label = np.argmax(np.bincount(target_label_tmp[pid_index]))
        num_correct += (target_label_tmp[pid_index] == pred_label).astype(np.float32).sum()
    cluster_accuracy = num_correct / len(target_label_tmp)
    print(f'cluster accucary: {cluster_accuracy:.3f}')


class uncer(object):
    def __init__(self):
        self.sm = torch.nn.Softmax(dim=1)
        self.log_sm = torch.nn.LogSoftmax(dim=1)
        # self.cross_batch=CrossBatchMemory()
        self.kl_distance = nn.KLDivLoss(reduction='none')
    def kl_cal(self,pred1,pred1_ema):
        variance = torch.sum(self.kl_distance(self.log_sm(pred1),
                                              self.sm(pred1_ema.detach())), dim=1)
        exp_variance = torch.exp(-variance)
        return exp_variance


def main_worker(args):
    global start_epoch, best_mAP

    cudnn.benchmark = True

    sys.stdout = Logger(osp.join(args.logs_dir, 'log.txt'))
    print("==========\nArgs:{}\n==========".format(args))

    # Create data loaders
    iters = args.iters if (args.iters > 0) else None
    ncs = [int(x) for x in args.ncs.split(',')]
    # ncs_dbscan=ncs.copy()
    dataset_target, label_dict = get_data(args.dataset_target, args.data_dir, len(ncs))
    test_loader_target = get_test_loader(dataset_target, args.height, args.width, args.batch_size, args.workers)
    tar_cluster_loader = get_test_loader(dataset_target, args.height, args.width, args.batch_size, args.workers,
                                         testset=dataset_target.train)


    dataset_source, _ = get_data(args.dataset_source, args.data_dir, len(ncs))
    sour_cluster_loader = get_test_loader(dataset_source, args.height, args.width, args.batch_size, args.workers,
                                          testset=dataset_source.train)
    train_loader_source = get_train_loader(dataset_source, args.height, args.width, 0, args.batch_size, args.workers,
                                           args.num_instances, args.iters, dataset_source.train)

    source_classes = dataset_source.num_train_pids
    distribution,_ = write_sta_im(dataset_source.train)

    fc_len = 3500
    model_1, _, model_1_ema, _ = create_model(args, [fc_len for _ in range(len(ncs))])
    # print(model_1)

    epoch = 0
    target_features_dict, _ = extract_features(model_1_ema, tar_cluster_loader, print_freq=100)
    target_features = F.normalize(torch.stack(list(target_features_dict.values())), dim=1)

    # Calculate distance
    print('==> Create pseudo labels for unlabeled target domain')

    rerank_dist = compute_jaccard_distance(target_features, k1=args.k1, k2=args.k2)
    del target_features
    if (epoch == 0):
        # DBSCAN cluster
        eps = 0.6  # 0.6
        print('Clustering criterion: eps: {:.3f}'.format(eps))
        cluster = DBSCAN(eps=eps, min_samples=4, metric='precomputed', n_jobs=-1)

    # select & cluster images as training set of this epochs
    pseudo_labels = cluster.fit_predict(rerank_dist)

    # num_ids = len(set(pseudo_labels)) - (1 if -1 in pseudo_labels else 0)
    plabel=[]
    new_dataset=[]
    for i, (item, label) in enumerate(zip(dataset_target.train, pseudo_labels)):

        if label == -1:
            continue
        plabel.append(label)
        new_dataset.append((item[0], label, item[-1]))

    target_label = [plabel]
    ncs = [len(set(plabel)) + 1]
    print('new class are {}, length of new dataset is {}'.format(ncs, len(new_dataset)))
    model_1.module.classifier0_3500 = nn.Linear(2048, ncs[0]+source_classes, bias=False).cuda()
    model_1_ema.module.classifier0_3500 = nn.Linear(2048, ncs[0]+source_classes, bias=False).cuda()

    model_1.module.classifier3_0_3500 = nn.Linear(1024, ncs[0]+source_classes, bias=False).cuda()
    model_1_ema.module.classifier3_0_3500 = nn.Linear(1024, ncs[0]+source_classes, bias=False).cuda()
    print(model_1.module.classifier0_3500)
    # if epoch !=0:
    #     model_1.module.classifier0_3500.weight.data.copy_(torch.from_numpy(normalize(target_centers,axis=1)).float().cuda())
    #     model_1_ema.module.classifier0_3500.weight.data.copy_(torch.from_numpy(normalize(target_centers,axis=1)).float().cuda())



    # Initialize source-domain class centroids
    print("==> Initialize source-domain class centroids in the hybrid memory")
    source_features, _ = extract_features(model_1, sour_cluster_loader, print_freq=50)
    sour_fea_dict = collections.defaultdict(list)
    print("==> Ending source-domain class centroids in the hybrid memory")
    for f, pid, _ in sorted(dataset_source.train):
        sour_fea_dict[pid].append(source_features[f].unsqueeze(0))
    source_centers = [torch.cat(sour_fea_dict[pid], 0).mean(0) for pid in sorted(sour_fea_dict.keys())]
    source_centers = torch.stack(source_centers, 0)
    source_centers = F.normalize(source_centers, dim=1)
    del sour_fea_dict, source_features, sour_cluster_loader

    # Evaluator
    evaluator_1 = Evaluator(model_1)
    evaluator_1_ema = Evaluator(model_1_ema)


    clusters = [args.num_clusters] * args.epochs# TODO: dropout clusters

    k_memory=8192
    contrast = onlinememory(2048, len(new_dataset),sour_numclass=source_classes,K=k_memory+source_classes,
                             index2label=target_label, choice_c=args.choice_c, T=0.07,
                             use_softmax=True).cuda()
    contrast.index_memory = torch.cat((torch.arange(source_classes), -1*torch.ones(k_memory).long()), dim=0).cuda()
    contrast.memory = torch.cat((source_centers, torch.rand(k_memory, 2048)), dim=0).cuda()

    tar_selflabel_loader = get_test_loader(dataset_target, args.height, args.width, args.batch_size, args.workers,
                                         testset=new_dataset)

    o = Optimizer(target_label, dis_gt=distribution, m=model_1, ncl=ncs,
                  t_loader=tar_selflabel_loader, N=len(new_dataset), fc_len=fc_len)

    uncertainty=collections.defaultdict(list)
    print("Training begining~~~~~~!!!!!!!!!")
    for epoch in range(len(clusters)):

        iters_ = 300 if epoch  % 1== 0 else iters
        if epoch % 6 == 0 and epoch !=0:
            target_features_dict, _ = extract_features(model_1_ema, tar_cluster_loader, print_freq=50)

            target_features = torch.stack(list(target_features_dict.values()))
            target_features = F.normalize(target_features, dim=1)

            print('==> Create pseudo labels for unlabeled target domain with')
            rerank_dist = compute_jaccard_distance(target_features, k1=args.k1, k2=args.k2)

            # select & cluster images as training set of this epochs
            pseudo_labels = cluster.fit_predict(rerank_dist)
            plabel = []

            new_dataset = []

            for i, (item, label) in enumerate(zip(dataset_target.train, pseudo_labels)):
                if label == -1:continue
                plabel.append(label)
                new_dataset.append((item[0], label, item[-1]))

            target_label = [plabel]
            ncs = [len(set(plabel)) + 1]

            tar_selflabel_loader = get_test_loader(dataset_target, args.height, args.width, args.batch_size, args.workers,
                                                 testset=new_dataset)
            o = Optimizer(target_label, dis_gt=distribution, m=model_1, ncl=ncs,
                          t_loader=tar_selflabel_loader, N=len(new_dataset),fc_len=fc_len)


            contrast.index_memory = torch.cat((torch.arange(source_classes), -1 * torch.ones(k_memory).long()),
                                              dim=0).cuda()

            model_1.module.classifier0_3500 = nn.Linear(2048, ncs[0] + source_classes, bias=False).cuda()
            model_1_ema.module.classifier0_3500 = nn.Linear(2048, ncs[0] + source_classes, bias=False).cuda()

            model_1.module.classifier3_0_3500 = nn.Linear(1024, ncs[0] + source_classes, bias=False).cuda()
            model_1_ema.module.classifier3_0_3500 = nn.Linear(1024, ncs[0] + source_classes, bias=False).cuda()
            print(model_1.module.classifier0_3500)
            # if epoch !=0:
            #     model_1.module.classifier0_3500.weight.data.copy_(torch.from_numpy(normalize(target_centers,axis=1)).float().cuda())
            #     model_1_ema.module.classifier0_3500.weight.data.copy_(torch.from_numpy(normalize(target_centers,axis=1)).float().cuda())

        target_label_o = o.L
        target_label = [list(np.asarray(target_label_o[0].data.cpu())+source_classes)]
        contrast.index2label = [[i for i in range(source_classes)] + target_label[0]]

        # change pseudo labels
        for i in range(len(new_dataset)):
            new_dataset[i] = list(new_dataset[i])
            for j in range(len(ncs)):
                new_dataset[i][j+1] = int(target_label[j][i])
            new_dataset[i] = tuple(new_dataset[i])

        cc=args.choice_c#(args.choice_c+1)%len(ncs)
        train_loader_target = get_train_loader(dataset_target, args.height, args.width, cc,
                                               args.batch_size, args.workers, args.num_instances, iters_, new_dataset)

        # Optimizer
        params = []
        flag = 1.0
        # if 20<epoch<=40 or 60<epoch<=80 or 120<epoch:
        #     flag=0.1
        # else:
        #     flag=1.0

        for key, value in model_1.named_parameters():
            if not value.requires_grad:
                print(key)
                continue
            params += [{"params": [value], "lr": args.lr*flag, "weight_decay": args.weight_decay}]

        optimizer = torch.optim.Adam(params)

        # Trainer
        trainer = DbscanBaseTrainer(model_1, model_1_ema, contrast,
                             num_cluster=ncs, alpha=args.alpha, fc_len=fc_len)


        train_loader_target.new_epoch()
        train_loader_source.new_epoch()


        trainer.train(epoch, train_loader_target, train_loader_source, optimizer, args.choice_c,
                                    print_freq=args.print_freq, train_iters=iters_)

        def save_model(model_ema, is_best, best_mAP, mid):
            save_checkpoint({
                'state_dict': model_ema.state_dict(),
                'epoch': epoch + 1,
                'best_mAP': best_mAP,
            }, is_best, fpath=osp.join(args.logs_dir, 'model' + str(mid) + '_checkpoint.pth.tar'))
        if epoch==20:
            args.eval_step=2
        elif epoch==40:
            args.eval_step=1
        if ((epoch + 1) % args.eval_step == 0 or (epoch == args.epochs - 1)):
            mAP_1 = 0#evaluator_1.evaluate(test_loader_target, dataset_target.query, dataset_target.gallery,
                     #          cmc_flag=False)


            mAP_2 = evaluator_1_ema.evaluate(test_loader_target, dataset_target.query, dataset_target.gallery,
                                             cmc_flag=False)
            is_best = (mAP_1 > best_mAP) or (mAP_2 > best_mAP)
            best_mAP = max(mAP_1, mAP_2, best_mAP)
            save_model(model_1, (is_best), best_mAP, 1)
            save_model(model_1_ema, (is_best and (mAP_1 <= mAP_2)), best_mAP, 2)

            print('\n * Finished epoch {:3d}  model no.1 mAP: {:5.1%} model no.2 mAP: {:5.1%}  best: {:5.1%}{}\n'.
                  format(epoch, mAP_1, mAP_2, best_mAP, ' *' if is_best else ''))

In [3]:
args = parser.parse_args('')

if args.seed is not None:
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    cudnn.deterministic = True

global start_epoch, best_mAP

cudnn.benchmark = True

sys.stdout = Logger(osp.join(args.logs_dir, 'log.txt'))
print("==========\nArgs:{}\n==========".format(args))

# Create data loaders
iters = args.iters if (args.iters > 0) else None
ncs = [int(x) for x in args.ncs.split(',')]
# ncs_dbscan=ncs.copy()
dataset_target, label_dict = get_data(args.dataset_target, args.data_dir, len(ncs))
test_loader_target = get_test_loader(dataset_target, args.height, args.width, args.batch_size, args.workers)
tar_cluster_loader = get_test_loader(dataset_target, args.height, args.width, args.batch_size, args.workers,
                                        testset=dataset_target.train)
# img_data, path, pid, cid, img_id_in_batch 


dataset_source, _ = get_data(args.dataset_source, args.data_dir, len(ncs))
sour_cluster_loader = get_test_loader(dataset_source, args.height, args.width, args.batch_size, args.workers,
                                        testset=dataset_source.train)
train_loader_source = get_train_loader(dataset_source, args.height, args.width, 0, args.batch_size, args.workers,
                                        args.num_instances, args.iters, dataset_source.train)

source_classes = dataset_source.num_train_pids
distribution,_ = write_sta_im(dataset_source.train)

Args:Namespace(dataset_source='market1501', dataset_target='dukemtmc', batch_size=64, workers=16, choice_c=0, num_clusters=700, ncs='60', k1=30, k2=6, height=256, width=128, num_instances=4, arch='ResNet50', features=0, dropout=0, lr=0.00035, momentum=0.9, alpha=0.999, moving_avg_momentum=0, weight_decay=0.0005, soft_ce_weight=0.5, soft_tri_weight=0.8, epochs=200, iters=200, lambda_value=0, rr_gpu=False, init_1='logs/market1501TOdukemtmc/market_pretrain_resnet/model_best.pth.tar', seed=1, print_freq=50, eval_step=5, n_jobs=16, data_dir='/home/jun/ReID_Dataset', logs_dir='/home/jun/logs/dbscan-market1501TOdukemtmc/hrnet_sbs_mb', lambda_tri=1.0, lambda_reg=1.0, lambda_ct=0.05, uncer_mode=0)
This dataset has been downloaded.
=> DukeMTMC-reID loaded
Dataset statistics:
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |

In [4]:
dataset_source.num_train_pids, dataset_source.num_query_pids, dataset_source.num_gallery_pids 

(751, 750, 751)

In [5]:
dataset_target.num_train_pids, dataset_target.num_query_pids, dataset_target.num_gallery_pids 

(702, 702, 1110)

In [6]:
img1, img2, path, pid, cid, id= next(iter(train_loader_source.loader))

In [7]:
fc_len = 3500
model_1, _, model_1_ema, _ = create_model(args, [fc_len for _ in range(len(ncs))])
# print(model_1)

Creating ResNet50 model with 751 source classes
GeneralizedMeanPoolingP
Creating ResNet50 model with 751 source classes
GeneralizedMeanPoolingP
=> Loaded checkpoint 'logs/market1501TOdukemtmc/market_pretrain_resnet/model_best.pth.tar'
missing keys in state_dict: {'classifier0_3500.weight'}
missing keys in state_dict: {'classifier0_3500.weight'}
load pretrain model:logs/market1501TOdukemtmc/market_pretrain_resnet/model_best.pth.tar
Extract Features: [100/259]	Time 0.045 (0.064)	Data 0.000 (0.005)	
Extract Features: [200/259]	Time 0.037 (0.054)	Data 0.000 (0.003)	
==> Create pseudo labels for unlabeled target domain
Computing jaccard distance...
Jaccard distance computing time cost: 17.35665202140808
Clustering criterion: eps: 0.600
new class are [889], length of new dataset is 14832
new class are [889], length of new dataset is 14832
torch.Size([32, 2048])
==> Initialize source-domain class centroids in the hybrid memory
Extract Features: [50/203]	Time 0.057 (0.123)	Data 0.000 (0.010)	


In [8]:
from torchinfo import summary
summary(model=model_1, 
        input_size=(32, 3, 256, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
       row_settings=["var_names"]
    )
    # Sample data

Layer (type (var_name))                                 Input Shape          Output Shape         Param #              Trainable
DataParallel (DataParallel)                             [32, 3, 256, 128]    [32, 2048]           --                   True
├─ResNet50 (module)                                     [32, 3, 256, 128]    [32, 2048]           11,372,544           True
│    └─ResNet (model)                                   --                   --                   2,049,000            True
│    │    └─Conv2d (conv1)                              [32, 3, 256, 128]    [32, 64, 128, 64]    9,408                True
│    │    └─DSBN2d (bn1)                                [32, 64, 128, 64]    [32, 64, 128, 64]    256                  True
│    │    └─ReLU (relu)                                 [32, 64, 128, 64]    [32, 64, 128, 64]    --                   --
│    │    └─MaxPool2d (maxpool)                         [32, 64, 128, 64]    [32, 64, 64, 32]     --                   --
│    │ 

In [8]:
epoch = 0
target_features_dict, _ = extract_features(model_1_ema, tar_cluster_loader, print_freq=100)
target_features = F.normalize(torch.stack(list(target_features_dict.values())), dim=1)

# Calculate distance
print('==> Create pseudo labels for unlabeled target domain')

rerank_dist = compute_jaccard_distance(target_features, k1=args.k1, k2=args.k2)

del target_features
if (epoch == 0):
    # DBSCAN cluster
    eps = 0.6  # 0.6
    print('Clustering criterion: eps: {:.3f}'.format(eps))
    cluster = DBSCAN(eps=eps, min_samples=4, metric='precomputed', n_jobs=-1)

# select & cluster images as training set of this epochs
pseudo_labels = cluster.fit_predict(rerank_dist)

bruteForceKnn is deprecated; call bfKnn instead


In [9]:
torch.stack(list(target_features_dict.values())).shape

torch.Size([16522, 2048])

In [10]:
# num_ids = len(set(pseudo_labels)) - (1 if -1 in pseudo_labels else 0)
plabel=[]
new_dataset=[]
for i, (item, label) in enumerate(zip(dataset_target.train, pseudo_labels)):
    if label == -1:
        continue
    plabel.append(label)
    new_dataset.append((item[0], label, item[-1])) # path, psudo_label, cid


In [13]:
target_label = [plabel]
ncs = [len(set(plabel)) + 1]
print('new class are {}, length of new dataset is {}'.format(ncs, len(new_dataset)))
model_1.module.classifier0_3500 = nn.Linear(2048, ncs[0]+source_classes, bias=False).cuda()
model_1_ema.module.classifier0_3500 = nn.Linear(2048, ncs[0]+source_classes, bias=False).cuda()

model_1.module.classifier3_0_3500 = nn.Linear(1024, ncs[0]+source_classes, bias=False).cuda()
model_1_ema.module.classifier3_0_3500 = nn.Linear(1024, ncs[0]+source_classes, bias=False).cuda()

In [14]:
from torchinfo import summary
summary(model=model_1, 
        input_size=(32, 3, 256, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
       row_settings=["var_names"]
    )
    # Sample data

Layer (type (var_name))                                 Input Shape          Output Shape         Param #              Trainable
DataParallel (DataParallel)                             [32, 3, 256, 128]    [32, 2048]           --                   True
├─ResNet50 (module)                                     [32, 3, 256, 128]    [32, 2048]           9,242,624            True
│    └─ResNet (model)                                   --                   --                   2,049,000            True
│    │    └─Conv2d (conv1)                              [32, 3, 256, 128]    [32, 64, 128, 64]    9,408                True
│    │    └─DSBN2d (bn1)                                [32, 64, 128, 64]    [32, 64, 128, 64]    256                  True
│    │    └─ReLU (relu)                                 [32, 64, 128, 64]    [32, 64, 128, 64]    --                   --
│    │    └─MaxPool2d (maxpool)                         [32, 64, 128, 64]    [32, 64, 64, 32]     --                   --
│    │ 

In [15]:
from torch.autograd import Variable
input = Variable(torch.FloatTensor(32, 3, 256, 128)).to('cpu')
output = model_1(input)
print(output.shape)

In [16]:
# Initialize source-domain class centroids
print("==> Initialize source-domain class centroids in the hybrid memory")
source_features, _ = extract_features(model_1, sour_cluster_loader, print_freq=50) 

sour_fea_dict = collections.defaultdict(list)

In [ ]:
source_features

OrderedDict([('/home/jun/ReID_Dataset/Market-1501-v15.09.15/bounding_box_train/1049_c6s2_129993_01.jpg',
              tensor([-0.0035,  0.0060, -0.0207,  ..., -0.0112, -0.0173,  0.0461])),
             ('/home/jun/ReID_Dataset/Market-1501-v15.09.15/bounding_box_train/1443_c5s3_057415_02.jpg',
              tensor([-0.0020, -0.0002, -0.0120,  ..., -0.0069, -0.0093, -0.0278])),
             ('/home/jun/ReID_Dataset/Market-1501-v15.09.15/bounding_box_train/0575_c6s2_004118_02.jpg',
              tensor([-0.0181, -0.0080,  0.0450,  ...,  0.0325, -0.0193,  0.0008])),
             ('/home/jun/ReID_Dataset/Market-1501-v15.09.15/bounding_box_train/1338_c6s3_065667_01.jpg',
              tensor([ 0.0107,  0.0307,  0.0204,  ...,  0.0096, -0.0114,  0.0327])),
             ('/home/jun/ReID_Dataset/Market-1501-v15.09.15/bounding_box_train/0597_c3s2_015437_02.jpg',
              tensor([-0.0119, -0.0087, -0.0081,  ...,  0.0795,  0.0050,  0.0316])),
             ('/home/jun/ReID_Dataset/Market-1501-

In [ ]:
source_features['/home/jun/ReID_Dataset/Market-1501-v15.09.15/bounding_box_train/1049_c6s2_129993_01.jpg'].unsqueeze(0).shape

torch.Size([1, 2048])

In [ ]:
for f, pid, _ in sorted(dataset_source.train):
    sour_fea_dict[pid].append(source_features[f].unsqueeze(0)) # [pid, tensor(1,2048)]

In [ ]:
source_centers = [torch.cat(sour_fea_dict[pid], 0).mean(0) for pid in sorted(sour_fea_dict.keys())] # Calculate mean values of each pid in source features
source_centers = torch.stack(source_centers, 0)
source_centers = F.normalize(source_centers, dim=1)
del sour_fea_dict, source_features, sour_cluster_loader

In [ ]:
# Evaluator
evaluator_1 = Evaluator(model_1)
evaluator_1_ema = Evaluator(model_1_ema)

In [ ]:
clusters = [args.num_clusters] * args.epochs
clusters[0]

700

In [ ]:
k_memory=8192
contrast = onlinememory(2048, len(new_dataset),sour_numclass=source_classes,K=k_memory+source_classes,
                             index2label=target_label, choice_c=args.choice_c, T=0.07,
                             use_softmax=True).cuda()
contrast.index_memory = torch.cat((torch.arange(source_classes), -1*torch.ones(k_memory).long()), dim=0).cuda()
contrast.memory = torch.cat((source_centers, torch.rand(k_memory, 2048)), dim=0).cuda()

tar_selflabel_loader = get_test_loader(dataset_target, args.height, args.width, args.batch_size, args.workers,
                                         testset=new_dataset)

o = Optimizer(target_label, dis_gt=distribution, m=model_1, ncl=ncs,
                  t_loader=tar_selflabel_loader, N=len(new_dataset), fc_len=fc_len)

uncertainty=collections.defaultdict(list)

In [ ]:
contrast.memory.shape

torch.Size([8943, 2048])

In [ ]:
target_label_o = o.L
target_label = [list(np.asarray(target_label_o[0].data.cpu())+source_classes)]
contrast.index2label = [[i for i in range(source_classes)] + target_label[0]]

In [ ]:
# change pseudo labels
for i in range(len(new_dataset)):
    new_dataset[i] = list(new_dataset[i])
    for j in range(len(ncs)):
        new_dataset[i][j+1] = int(target_label[j][i])
    new_dataset[i] = tuple(new_dataset[i])

In [ ]:
cc=args.choice_c#(args.choice_c+1)%len(ncs)
train_loader_target = get_train_loader(dataset_target, args.height, args.width, cc,
                                      args.batch_size, args.workers, args.num_instances, 300, new_dataset)

In [ ]:
# Optimizer
params = []
flag = 1.0
# if 20<epoch<=40 or 60<epoch<=80 or 120<epoch:
#     flag=0.1
# else:
#     flag=1.0

for key, value in model_1.named_parameters():
    if not value.requires_grad:
        print(key)
        continue
    params += [{"params": [value], "lr": args.lr*flag, "weight_decay": args.weight_decay}]

optimizer = torch.optim.Adam(params)

# Trainer
trainer = DbscanBaseTrainer(model_1, model_1_ema, contrast,
                        num_cluster=ncs, alpha=args.alpha, fc_len=fc_len)

In [ ]:
train_loader_target.new_epoch()
train_loader_source.new_epoch()


#trainer.train(epoch, train_loader_target, train_loader_source, optimizer, args.choice_c,
                                    #print_freq=args.print_freq, train_iters=300)

IndentationError: unexpected indent (160644632.py, line 6)